In [ ]:
from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

font_path = '~/Desktop/fonts'
font_list = [join(font_path, f) for f in listdir(font_path) if isfile(join(font_path, f)) and f.find('.DS_Store') == -1]
font_list

In [ ]:
from string import ascii_lowercase, ascii_uppercase
from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

begin = 0xac00
end = 0xd7a3

char_list = [chr(begin+idx) for idx in range(end - begin + 1)] \
    + [x for x in (ascii_lowercase + ascii_uppercase)] + [str(x) for x in range(10)] + [x for x in '~!@#$%^&*()_+-=\`<>?,.;:\'\"[]{}\\|']
print(len(char_list), char_list)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageFont, ImageDraw


def plot_img_inline(img):
    plt.figure()
    plt.imshow(np.asarray(img))
    plt.show()

    
def createFontSizes(font, size, char_list=char_list):
    imFont = ImageFont.truetype(font=font, size=size)
    return {char: imFont.getsize(char) for char in char_list}


font_sizes = [10, 20, 30, 50, 100]
font = font_list[0]

result = { font: { size: createFontSizes(font, size, char_list=char_list) for size in font_sizes }  for font in font_list }


MINIMUM_SAMPLE = 10
TARGET_SAMPLE_CHAR = len(font_sizes) * len(font) * MINIMUM_SAMPLE


print(font)
im = Image.new('RGB', (800, 600), color=(256, 256, 256))
draw = ImageDraw.Draw(im)
imFont = ImageFont.truetype(font=font, size=10)
fntSize = imFont.getsize(char_list[0])
draw.text((10, 10), char_list[0], font=imFont, fill=(0, 0, 0))

print(fntSize)

plot_img_inline(im)
imChar = im.crop((10 - 1, 10 - 1, 10 + fntSize[0] + 1, 10 + fntSize[1] + 1))
plot_img_inline(imChar)

In [ ]:
for k, v in result.items():
    print(k)

In [ ]:
im = Image.new('RGB', (800, 600), color=(255, 255, 255))
draw = ImageDraw.Draw(im)
imFont = ImageFont.truetype(font=font, size=10)
fntSize = imFont.getsize(char_list[0])
draw.text((10, 10), char_list[0], font=imFont, fill=(0, 0, 0))
imChar = im.crop((10, 10, 10 + fntSize[0], 10 + fntSize[1]))
print(fntSize)

in_data = np.asarray(imChar, dtype=np.uint8)

print(in_data)

# im


In [183]:
'''
# Generate image with random string & font & size
1. select font
1. select size
1. select chars 2 ~ 10
1. pick random position
1. check fill_image_ratio

'''

import random
import copy
from string import ascii_lowercase, ascii_uppercase
from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

begin = 0xac00
end = 0xd7a3

class Sample_IMG():
    def __init__(self, n_char=10, fill_ratio=0.5, width=800, height=600):
        self.n_char = n_char
        self.fill_ratio = fill_ratio
        self.width = width
        self.height = height
        pass
    
    def generate():
        pass
    

class CH_Dataset():
    def __init__(self, font_path=None):
        self.char_list = [chr(begin + idx) for idx in range(end - begin + 1)] \
                         + [x for x in (ascii_lowercase + ascii_uppercase)] + [str(x) for x in range(10)] + [x for x in
                                                                                                             '~!@#$%^&*()_+-=<>?,.;:[]{}|']

        self.char_list = self.char_list[:30]
        self.font_list = [join(font_path, f) for f in listdir(font_path) if
                          isfile(join(font_path, f)) and f.find('.DS_Store') == -1]

        self.font_sizes = [10] * 2 + [20] * 5 + [30] * 3 + [50] * 2 + [100]

        self.counter = 0
        self.gen_counter = 0
        self.fill_ratio = 0

        self.font_size_tuples = None
        self.char_list_list = None
        self.counter_list = None
        
        self.imFont = dict()
        
        self.fill_area = 0
        self.width = 0
        self.height = 0

    def isStop(self):
        self.counter = self.counter + 1
        if self.counter > 50:
            return True
        else:
            return False

    def hasFill(self):
        print('hasFill', self.fill_ratio * self.width * self.height, self.fill_area)
        if self.fill_ratio * self.width * self.height < self.fill_area:
            return True
        else:
            return False

    def getFont(self):
        if self.font_list is not None and len(self.font_list) > 0 and len(self.font_sizes) > 0:
            self.gen_counter += 1
            return self.font_list[self.gen_counter % len(self.font_list)], self.font_sizes[
                self.gen_counter % len(self.font_sizes)]
        else:
            return None, None
        
    def getIMFont(self, font, size):
        if font not in self.imFont:
            self.imFont[font] = dict()
            
        if size not in self.imFont[font]:
            self.imFont[font][size] = ImageFont.truetype(font=font, size=size)
        return ImageFont.truetype(font=font, size=size)
    
        return self.imFont[font][size]

    def getChars(self):
        print('self.counter_list', self.counter_list)

        # pick a random char list
        idx = random.randrange(len(self.char_list_list))

        # split first k chars(1~10)
        k_chars = random.randrange(10) + 1

        # get fist k chars
        current_char_idx = self.counter_list[idx]

        # make sure target_idx <= len(self.char_list_list[idx])
        target_idx = current_char_idx + k_chars if current_char_idx + k_chars < len(self.char_list_list[idx]) else len(
            self.char_list_list[idx])

        print(idx, current_char_idx, target_idx, self.char_list_list[idx][current_char_idx:target_idx])
        # pick chars (current_char_idx:target_idx)
        sample_chars = self.char_list_list[idx][current_char_idx:target_idx]
        self.counter_list[idx] = target_idx
        print('self.counter_list', self.counter_list)
        return idx, self.char_list_list[idx][current_char_idx:target_idx]
    
    def hasMoreData(self):
        filled_counter = sum([counter >= len(char_list) for counter, char_list in zip(self.counter_list, self.char_list_list)])
        
        print(filled_counter, [counter >= len(char_list) for counter, char_list in zip(self.counter_list, self.char_list_list)])
        if filled_counter >= len(self.counter_list):
            return False
        else:
            return True

    def generate(self, n_char=10, fill_ratio=0.5, width=800, height=600):
        '''
        generate single image sample containing multiple chars

        :param n_char: number of char per font & size
        :param ratio: fill area ratio of image with character region
        :param width: width of image
        :param height: height of image
        :return:
        '''
        pass

        if self.font_size_tuples is None or self.char_list_list is None or self.counter_list is None:
            return None

        self.fill_area = 0
        self.width = width
        self.height = height
        self.fill_ratio = fill_ratio
        
        while self.hasFill() is False and self.hasMoreData() is True:
            tuple_idx, chars = self.getChars()
            (font, size) = self.font_size_tuples[tuple_idx]
            
            imFont = self.getIMFont(font, size)
            
            char_sizes = result[font][size]
            char_sizes = [imFont.getsize(ch) for ch in chars]
            sum_area = sum([ch_size[0] * ch_size[1] for ch_size in char_sizes])
            self.fill_area += sum_area

            print(font, size, char_sizes, sum_area, chars)
            
    def generateSamples(self, n_char=10, fill_ratio=0.5, width=800, height=600):
        '''
        generate samples:
            1. generate (font, size) tuple
            1. create N char list(N = # of (font, size) tuple)
            1. shuffle each char list

        :param n_char: number of char per font & size
        :param ratio: fill area ratio of image with character region
        :param width: width of image
        :param height: height of image

        :return:
        '''
        self.font_size_tuples = [(f, s) for f in self.font_list for s in self.font_sizes][:5]
        self.char_list_list = [copy.deepcopy(self.char_list) for _ in range(len(self.font_size_tuples))]
        [random.shuffle(ch_list) for ch_list in self.char_list_list]
        self.counter_list = [0] * len(self.char_list_list)

        results = list()
        while self.isStop() is False:
            gen_img = self.generate(n_char=n_char, fill_ratio=fill_ratio, width=width, height=height)
            if gen_img:
                results.append(gen_img)
        return results


chd = CH_Dataset(font_path='/Users/ali.jeon/Desktop/fonts')
chd.generateSamples()


hasFill 240000.0 0
0 [False, False, False, False, False]
self.counter_list [0, 0, 0, 0, 0]
1 0 1 ['갇']
self.counter_list [0, 1, 0, 0, 0]
/Users/ali.jeon/Desktop/fonts/BM-HANNA.ttf 10 [(9, 10)] 90 ['갇']
hasFill 240000.0 90
0 [False, False, False, False, False]
self.counter_list [0, 1, 0, 0, 0]
2 0 6 ['갋', '개', '객', '갉', '값', '갆']
self.counter_list [0, 1, 6, 0, 0]
/Users/ali.jeon/Desktop/fonts/BM-HANNA.ttf 20 [(17, 16), (17, 18), (17, 19), (17, 19), (17, 18), (17, 16)] 1802 ['갋', '개', '객', '갉', '값', '갆']
hasFill 240000.0 1892
0 [False, False, False, False, False]
self.counter_list [0, 1, 6, 0, 0]
2 6 15 ['갏', '갈', '갎', '갚', '갅', '갗', '갊', '갇', '같']
self.counter_list [0, 1, 15, 0, 0]
/Users/ali.jeon/Desktop/fonts/BM-HANNA.ttf 20 [(17, 16), (17, 19), (17, 16), (17, 18), (17, 16), (17, 19), (17, 19), (17, 18), (17, 18)] 2703 ['갏', '갈', '갎', '갚', '갅', '갗', '갊', '갇', '같']
hasFill 240000.0 4595
0 [False, False, False, False, False]
self.counter_list [0, 1, 15, 0, 0]
3 0 2 ['갘', '감']
self.count

[]